# [E-12 다음에 볼 영화 예측하기]

시간이 부족해서 깔끔하게 정리하지 못한점,, 미리 사과 드립니다.

## 1. 데이터 불러오기

In [2]:
import datetime as dt
from pathlib import Path
import os

import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [27]:
data_path = Path(os.getenv('HOME')+'/aiffel/yoochoose/data/') 
train_path = data_path / 'ratings.dat'

def load_data(data_path: Path, nrows=None):
    data = pd.read_csv(data_path, sep='::', header=None, usecols=[0, 1, 2, 3], dtype={0: np.int32, 1: np.int32, 2: np.int32}, nrows=nrows)
    data.columns = ['UserId', 'ItemId', 'Rating', 'Time']
    return data

data = load_data(train_path, None)
data.sort_values(['UserId', 'Time'], inplace=True)  # data를 id와 시간 순서로 정렬해줍니다.
data

,UserId,ItemId,Rating,Time
31,1,3186,4,978300019
22,1,1270,5,978300055
27,1,1721,4,978300055
37,1,1022,5,978300055
24,1,2340,3,978300103
...,...,...,...,...
1000019,6040,2917,4,997454429
999988,6040,1921,4,997454464
1000172,6040,1784,3,997454464
1000167,6040,161,3,997454486


추천 시스템을 구축할때 가장 먼저 확인해 볼 것은 유저수와 아이템 수이다.

In [28]:
print(data['UserId'].nunique(), data['ItemId'].nunique())

6040 3706


유저 수는 6040명, 아이템 숫자는 3706개이다.

## 2. 데이터 전처리

### 2.1 Session ID
#### shift()를 이용한 session id 정의

In [4]:
from datetime import datetime
data['Time'] = data['Time'].map(lambda x: datetime.fromtimestamp(x))
data['delta'] = (data['Time'] - data['Time'].shift()).fillna(pd.Timedelta(seconds=0))
data

,UserId,ItemId,Rating,Time,delta
31,1,3186,4,2000-12-31 22:00:19,0 days 00:00:00
22,1,1270,5,2000-12-31 22:00:55,0 days 00:00:36
27,1,1721,4,2000-12-31 22:00:55,0 days 00:00:00
37,1,1022,5,2000-12-31 22:00:55,0 days 00:00:00
24,1,2340,3,2000-12-31 22:01:43,0 days 00:00:48
...,...,...,...,...,...
1000019,6040,2917,4,2001-08-10 14:40:29,0 days 00:00:31
999988,6040,1921,4,2001-08-10 14:41:04,0 days 00:00:35
1000172,6040,1784,3,2001-08-10 14:41:04,0 days 00:00:00
1000167,6040,161,3,2001-08-10 14:41:26,0 days 00:00:22


In [24]:
from datetime import timedelta
session_id = 0
exuser = data.UserId[0]
a = timedelta(minutes=30)

def get_session_id(x) :
    global session_id
    global exuser 
    if (exuser != x['UserId']) or (x['delta'] > a):
        session_id +=1
    exuser = x['UserId']
    return session_id

In [25]:
data['SessionId'] = data.apply(get_session_id, axis=1)
data

,UserId,ItemId,Rating,Time,delta,SessionId
31,1,3186,4,2000-12-31 22:00:19,0 days 00:00:00,0
22,1,1270,5,2000-12-31 22:00:55,0 days 00:00:36,0
27,1,1721,4,2000-12-31 22:00:55,0 days 00:00:00,0
37,1,1022,5,2000-12-31 22:00:55,0 days 00:00:00,0
24,1,2340,3,2000-12-31 22:01:43,0 days 00:00:48,0
...,...,...,...,...,...,...
1000019,6040,2917,4,2001-08-10 14:40:29,0 days 00:00:31,25161
999988,6040,1921,4,2001-08-10 14:41:04,0 days 00:00:35,25161
1000172,6040,1784,3,2001-08-10 14:41:04,0 days 00:00:00,25161
1000167,6040,161,3,2001-08-10 14:41:26,0 days 00:00:22,25161


### 2.2 Session Length

In [26]:
session_length = data.groupby('SessionId').size()
session_length

SessionId
0         40
1         13
2        129
3         51
4         21
        ... 
25158      7
25159     14
25160      1
25161     21
25162      1
Length: 25163, dtype: int64

In [27]:
session_length.median(), session_length.mean()

(6.0, 39.74919524698963)

In [35]:
session_length.min(), session_length.max()

(1, 1193)

In [36]:
session_length.quantile(0.999)

764.3520000000135

평균 세션의 길이는 39 이고, 99.9% 세션은 길이가 764이하 이다.  
그러나 최대 세션은 1193으로 아주 길다.. 한번 데이터를 살펴보자

In [37]:
long_session = session_length[session_length==1193].index[0]
data[data['SessionId']==long_session]

,UserId,ItemId,Rating,Time,delta,SessionId
637775,3841,1480,3,2000-08-11 11:57:39,0 days 21:58:11,16445
637808,3841,969,5,2000-08-11 11:57:39,0 days 00:00:00,16445
638269,3841,858,5,2000-08-11 11:57:39,0 days 00:00:00,16445
638716,3841,2019,5,2000-08-11 11:57:39,0 days 00:00:00,16445
638733,3841,1221,5,2000-08-11 11:57:39,0 days 00:00:00,16445
...,...,...,...,...,...,...
638038,3841,3806,5,2000-08-11 14:23:11,0 days 00:00:00,16445
638243,3841,1379,2,2000-08-11 14:23:11,0 days 00:00:00,16445
638459,3841,1928,4,2000-08-11 14:23:11,0 days 00:00:00,16445
638710,3841,2016,3,2000-08-11 14:23:11,0 days 00:00:00,16445


이 사람은 거의 2시간 반동안 엄첨 짧게 많은 것 클릭했다. 평점을 주는건데 이렇게 빠르게 많이 할 수있는가,,?  
이상하다고 생각해서 너무 긴 세션은 없애 주기로한다. (cleansing)

### 2.3 Session Time

In [38]:
oldest, latest = data['Time'].min(), data['Time'].max()
print(oldest) 
print(latest)

2000-04-25 23:05:32
2003-02-28 17:49:50


3년치 데이터가 있다! 그중 일년치 데이터만 써보자

In [40]:
type(latest)

pandas._libs.tslibs.timestamps.Timestamp

In [39]:
month_ago = latest - dt.timedelta(days=365)     # 최종 날짜로부터 365일 이전 날짜를 구한다.  
data = data[data['Time'] > month_ago]   # 방금 구한 날짜 이후의 데이터만 모은다. 
data

,UserId,ItemId,Rating,Time,delta,SessionId
5170,36,1387,5,2002-03-12 03:46:59,37 days 00:05:22,85
5267,36,1201,4,2002-03-12 03:46:59,0 days 00:00:00,85
5122,36,1291,5,2002-03-12 03:47:16,0 days 00:00:17,85
5123,36,2167,5,2002-03-12 03:48:25,0 days 00:01:09,85
5290,36,2951,4,2002-03-12 03:48:25,0 days 00:00:00,85
...,...,...,...,...,...,...
992358,5996,3835,3,2002-04-29 20:46:24,0 days 00:13:12,25050
992279,5996,2422,3,2002-04-29 20:47:05,0 days 00:00:41,25050
992702,5996,168,3,2002-09-03 13:12:26,126 days 16:25:21,25051
992459,5996,339,4,2002-10-07 13:24:39,34 days 00:12:13,25052


### 2.4 Data Cleansing

In [45]:
# short_session을 제거한 다음 unpopular item을 제거하면 다시 길이가 1인 session이 생길 수 있습니다.
# 이를 위해 반복문을 통해 지속적으로 제거 합니다.
# + 너무 긴 session도 제거 해주겠습니다.
def cleanse_recursive(data: pd.DataFrame, shortest, logest, least_click) -> pd.DataFrame:
    while True:
        before_len = len(data)
        data = cleanse_short_session(data, shortest)
        data = cleanse_long_session(data, logest)
        data = cleanse_unpopular_item(data, least_click)
        after_len = len(data)
        if before_len == after_len:
            break
    return data


def cleanse_short_session(data: pd.DataFrame, shortest):
    session_len = data.groupby('SessionId').size()
    session_use = session_len[session_len >= shortest].index
    data = data[data['SessionId'].isin(session_use)]
    return data

def cleanse_long_session(data: pd.DataFrame, logest):
    session_len = data.groupby('SessionId').size()
    session_use = session_len[session_len <= logest].index
    data = data[data['SessionId'].isin(session_use)]
    return data


def cleanse_unpopular_item(data: pd.DataFrame, least_click):
    item_popular = data.groupby('ItemId').size()
    item_use = item_popular[item_popular >= least_click].index
    data = data[data['ItemId'].isin(item_use)]
    return data

In [46]:
data = cleanse_recursive(data, shortest=2, logest=700, least_click=20) # 형준님, 진아님 감사합니다(least_click 올리기)
data

,UserId,ItemId,Rating,Time,delta,SessionId
5122,36,1291,5,2002-03-12 03:47:16,0 days 00:00:17,85
5359,36,2115,5,2002-03-12 03:48:25,0 days 00:00:00,85
5222,36,296,5,2002-12-22 08:16:59,0 days 00:05:34,88
5062,36,1266,5,2002-12-22 08:18:29,0 days 00:00:41,88
5076,36,595,4,2002-12-22 08:18:29,0 days 00:00:00,88
...,...,...,...,...,...,...
992246,5996,3039,4,2002-04-29 19:46:04,0 days 00:01:07,25050
992203,5996,3791,3,2002-04-29 19:58:39,0 days 00:00:25,25050
992571,5996,3247,4,2002-04-29 20:05:17,0 days 00:00:00,25050
992355,5996,3826,1,2002-04-29 20:10:17,0 days 00:00:45,25050


In [47]:
session_len = data.groupby('SessionId').size()
session_len[(session_len > 700)].index


Int64Index([], dtype='int64', name='SessionId')

### 2.5 Train/Valid/Test split

In [48]:
def split_by_date(data: pd.DataFrame, n_days: int):
    final_time = data['Time'].max()
    session_last_time = data.groupby('SessionId')['Time'].max()
    session_in_train = session_last_time[session_last_time < final_time - dt.timedelta(n_days)].index
    session_in_test = session_last_time[session_last_time >= final_time - dt.timedelta(n_days)].index

    before_date = data[data['SessionId'].isin(session_in_train)]
    after_date = data[data['SessionId'].isin(session_in_test)]
    after_date = after_date[after_date['ItemId'].isin(before_date['ItemId'])]
    return before_date, after_date

In [49]:
tr, test = split_by_date(data, n_days=30)
tr, val = split_by_date(tr, n_days=70)

In [50]:
# data에 대한 정보를 살펴봅니다.
def stats_info(data: pd.DataFrame, status: str):
    print(f'* {status} Set Stats Info\n'
          f'\t Events: {len(data)}\n'
          f'\t Sessions: {data["SessionId"].nunique()}\n'
          f'\t Items: {data["ItemId"].nunique()}\n'
          f'\t First Time : {data["Time"].min()}\n'
          f'\t Last Time : {data["Time"].max()}\n')

In [51]:
stats_info(tr, 'train')
stats_info(val, 'valid')
stats_info(test, 'test')

* train Set Stats Info
	 Events: 2856
	 Sessions: 429
	 Items: 153
	 First Time : 2002-02-28 19:06:39
	 Last Time : 2002-11-17 18:13:16

* valid Set Stats Info
	 Events: 713
	 Sessions: 115
	 Items: 152
	 First Time : 2002-11-17 20:56:25
	 Last Time : 2003-01-26 18:45:41

* test Set Stats Info
	 Events: 353
	 Sessions: 34
	 Items: 138
	 First Time : 2003-01-28 18:41:15
	 Last Time : 2003-02-27 18:15:10



In [52]:
# train set에 없는 아이템이 val, test기간에 생길 수 있으므로 train data를 기준으로 인덱싱합니다.
id2idx = {item_id : index for index, item_id in enumerate(tr['ItemId'].unique())}

def indexing(df, id2idx):
    df['item_idx'] = df['ItemId'].map(lambda x: id2idx.get(x, -1))  # id2idx에 없는 아이템은 모르는 값(-1) 처리 해줍니다.
    return df

tr = indexing(tr, id2idx)
val = indexing(val, id2idx)
test = indexing(test, id2idx)

In [53]:
save_path = data_path / 'processed'
save_path.mkdir(parents=True, exist_ok=True)

tr.to_pickle(save_path / 'train.pkl')
val.to_pickle(save_path / 'valid.pkl')
test.to_pickle(save_path / 'test.pkl')

## 3. Data pipeline

### 3.1 Session-Parllel Mini-Batches 

Session의 길이는 매우 짧은 것 들이 대부분이지만 긴 것도 있다.  
그래서 이 세션들을 데이터 샘플 하나로 보고 mini-batch를 구성하게 되면 젤 긴 세션의 연산이 끝날 때까지 짧은 세션들이 기다려야한다.  
그래서 Session-Parlle Mini-Batches를 쓰는데, 이름처럼 Session이 끝날 때까지 기다리지 않고 병렬적으로 계산한다.  
아래의 그림처럼 session2가 끝났다면 다음으로 session4가 시작한다.  
이렇게 구성했을 때 mini-batch의 shape는 (3,1,1)이 되고 RNN cell의 ste가 1개로만 이루어 진다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/input2.max-800x600.png)

### 3.2 Session Dataset

In [54]:
class SessionDataset:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, data):
        self.df = data
        self.click_offsets = self.get_click_offsets()
        self.session_idx = np.arange(self.df['SessionId'].nunique())  # indexing to SessionId

    def get_click_offsets(self):
        """
        Return the indexes of the first click of each session IDs,
        """
        offsets = np.zeros(self.df['SessionId'].nunique() + 1, dtype=np.int32)
        offsets[1:] = self.df.groupby('SessionId').size().cumsum()
        return offsets

In [55]:
tr_dataset = SessionDataset(tr)
tr_dataset.df.head(10)

,UserId,ItemId,Rating,Time,delta,SessionId,item_idx
5122,36,1291,5,2002-03-12 03:47:16,0 days 00:00:17,85,0
5359,36,2115,5,2002-03-12 03:48:25,0 days 00:00:00,85,1
9716,65,260,4,2002-05-27 02:58:29,40 days 22:46:43,153,2
9703,65,2628,4,2002-05-27 02:58:30,0 days 00:00:01,153,3
9745,65,1210,4,2002-05-27 02:58:30,0 days 00:00:00,153,4
13366,104,2858,5,2002-03-13 21:41:16,445 days 16:49:52,247,5
13330,104,2997,4,2002-03-13 21:41:40,0 days 00:00:13,247,6
18642,146,1639,4,2002-05-21 17:47:36,0 days 00:00:17,387,7
18588,146,780,3,2002-05-21 17:49:29,0 days 00:01:53,387,8
18820,146,1059,4,2002-05-21 17:55:55,0 days 00:00:00,387,9


In [56]:
tr_dataset.click_offsets

array([   0,    2,    5,    7,   16,   21,   23,   27,   33,   40,   45,
         48,   51,   55,   72,   81,   92,   99,  111,  113,  115,  118,
        120,  122,  125,  128,  139,  154,  159,  161,  163,  166,  189,
        199,  202,  204,  207,  211,  231,  252,  255,  257,  289,  296,
        316,  319,  321,  350,  352,  362,  365,  374,  376,  383,  394,
        398,  402,  406,  410,  414,  424,  427,  435,  445,  449,  455,
        478,  481,  483,  485,  487,  489,  491,  494,  499,  501,  504,
        517,  521,  546,  597,  603,  607,  614,  616,  618,  624,  626,
        632,  634,  637,  643,  645,  651,  653,  655,  657,  659,  661,
        663,  665,  669,  672,  678,  682,  684,  697,  706,  718,  724,
        726,  730,  737,  742,  744,  747,  749,  755,  758,  760,  762,
        765,  780,  782,  784,  786,  789,  794,  797,  802,  806,  808,
        810,  812,  816,  837,  839,  841,  845,  858,  870,  885,  887,
        889,  893,  895,  897,  899,  903,  906,  9

In [57]:
tr_dataset.session_idx

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

### 3.2 Session Data Loader

In [58]:
class SessionDataLoader:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, dataset: SessionDataset, batch_size=50):
        self.dataset = dataset
        self.batch_size = batch_size

    def __iter__(self):
        """ Returns the iterator for producing session-parallel training mini-batches.
        Yields:
            input (B,):  Item indices that will be encoded as one-hot vectors later.
            target (B,): a Variable that stores the target item indices
            masks: Numpy array indicating the positions of the sessions to be terminated
        """

        start, end, mask, last_session, finished = self.initialize()  # initialize 메소드에서 확인해주세요.
        """
        start : Index Where Session Start
        end : Index Where Session End
        mask : indicator for the sessions to be terminated
        """

        while not finished:
            min_len = (end - start).min() - 1  # Shortest Length Among Sessions
            for i in range(min_len):
                # Build inputs & targets
                inp = self.dataset.df['item_idx'].values[start + i]
                target = self.dataset.df['item_idx'].values[start + i + 1]
                yield inp, target, mask

            start, end, mask, last_session, finished = self.update_status(start, end, min_len, last_session, finished)

    def initialize(self):
        first_iters = np.arange(self.batch_size)    # 첫 배치에 사용할 세션 Index를 가져옵니다.
        last_session = self.batch_size - 1    # 마지막으로 다루고 있는 세션 Index를 저장해둡니다.
        start = self.dataset.click_offsets[self.dataset.session_idx[first_iters]]       # data 상에서 session이 시작된 위치를 가져옵니다.
        end = self.dataset.click_offsets[self.dataset.session_idx[first_iters] + 1]  # session이 끝난 위치 바로 다음 위치를 가져옵니다.
        mask = np.array([])   # session의 모든 아이템을 다 돌은 경우 mask에 추가해줄 것입니다.
        finished = False         # data를 전부 돌았는지 기록하기 위한 변수입니다.
        return start, end, mask, last_session, finished

    def update_status(self, start: np.ndarray, end: np.ndarray, min_len: int, last_session: int, finished: bool):  
        # 다음 배치 데이터를 생성하기 위해 상태를 update합니다.
        
        start += min_len   # __iter__에서 min_len 만큼 for문을 돌았으므로 start를 min_len 만큼 더해줍니다.
        mask = np.arange(self.batch_size)[(end - start) == 1]  
        # end는 다음 세션이 시작되는 위치인데 start와 한 칸 차이난다는 것은 session이 끝났다는 뜻입니다. mask에 기록해줍니다.

        for i, idx in enumerate(mask, start=1):  # mask에 추가된 세션 개수만큼 새로운 세션을 돌것입니다.
            new_session = last_session + i  
            if new_session > self.dataset.session_idx[-1]:  # 만약 새로운 세션이 마지막 세션 index보다 크다면 모든 학습데이터를 돈 것입니다.
                finished = True
                break
            # update the next starting/ending point
            start[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session]]     # 종료된 세션 대신 새로운 세션의 시작점을 기록합니다.
            end[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session] + 1]

        last_session += len(mask)  # 마지막 세션의 위치를 기록해둡니다.
        return start, end, mask, last_session, finished

In [59]:
tr_data_loader = SessionDataLoader(tr_dataset, batch_size=4)
tr_dataset.df.head(15)

,UserId,ItemId,Rating,Time,delta,SessionId,item_idx
5122,36,1291,5,2002-03-12 03:47:16,0 days 00:00:17,85,0
5359,36,2115,5,2002-03-12 03:48:25,0 days 00:00:00,85,1
9716,65,260,4,2002-05-27 02:58:29,40 days 22:46:43,153,2
9703,65,2628,4,2002-05-27 02:58:30,0 days 00:00:01,153,3
9745,65,1210,4,2002-05-27 02:58:30,0 days 00:00:00,153,4
13366,104,2858,5,2002-03-13 21:41:16,445 days 16:49:52,247,5
13330,104,2997,4,2002-03-13 21:41:40,0 days 00:00:13,247,6
18642,146,1639,4,2002-05-21 17:47:36,0 days 00:00:17,387,7
18588,146,780,3,2002-05-21 17:49:29,0 days 00:01:53,387,8
18820,146,1059,4,2002-05-21 17:55:55,0 days 00:00:00,387,9


In [60]:
iter_ex = iter(tr_data_loader)

In [61]:
inputs, labels, mask =  next(iter_ex)
print(f'Model Input Item Idx are : {inputs}')
print(f'Label Item Idx are : {"":5} {labels}')
print(f'Previous Masked Input Idx are {mask}')

Model Input Item Idx are : [0 2 5 7]
Label Item Idx are :       [1 3 6 8]
Previous Masked Input Idx are []


## 4. Modeling

In [62]:
def mrr_k(pred, truth: int, k: int):
    indexing = np.where(pred[:k] == truth)[0]
    if len(indexing) > 0:
        return 1 / (indexing[0] + 1)
    else:
        return 0


def recall_k(pred, truth: int, k: int) -> int:
    answer = truth in pred[:k]
    return int(answer)

In [63]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, GRU
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

In [64]:
def create_model(args):
    inputs = Input(batch_shape=(args.batch_size, 1, args.num_items))
    gru, _ = GRU(args.hsz, stateful=True, return_state=True, name='GRU')(inputs)
    dropout = Dropout(args.drop_rate)(gru)
    predictions = Dense(args.num_items, activation='softmax')(dropout)
    model = Model(inputs=inputs, outputs=[predictions])
    model.compile(loss=categorical_crossentropy, optimizer=Adam(args.lr), metrics=['accuracy'])
    model.summary()
    return model

In [74]:
class Args:
    def __init__(self, tr, val, test, batch_size, hsz, drop_rate, lr, epochs, k):
        self.tr = tr
        self.val = val
        self.test = test
        self.num_items = tr['ItemId'].nunique()
        self.num_sessions = tr['SessionId'].nunique()
        self.batch_size = batch_size
        self.hsz = hsz
        self.drop_rate = drop_rate
        self.lr = lr
        self.epochs = epochs
        self.k = k

args = Args(tr, val, test, batch_size=32, hsz=50, drop_rate=0.1, lr=0.001, epochs=30, k=20)

In [75]:
model = create_model(args)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(32, 1, 153)]            0         
_________________________________________________________________
GRU (GRU)                    [(32, 50), (32, 50)]      30750     
_________________________________________________________________
dropout_2 (Dropout)          (32, 50)                  0         
_________________________________________________________________
dense_2 (Dense)              (32, 153)                 7803      
Total params: 38,553
Trainable params: 38,553
Non-trainable params: 0
_________________________________________________________________


## 5. Train

In [76]:
# train 셋으로 학습하면서 valid 셋으로 검증합니다.
def train_model(model, args):
    train_dataset = SessionDataset(args.tr)
    train_loader = SessionDataLoader(train_dataset, batch_size=args.batch_size)

    for epoch in range(1, args.epochs + 1):
        total_step = len(args.tr) - args.tr['SessionId'].nunique()
        tr_loader = tqdm(train_loader, total=total_step // args.batch_size, desc='Train', mininterval=1)
        for feat, target, mask in tr_loader:
            reset_hidden_states(model, mask)  # 종료된 session은 hidden_state를 초기화합니다. 아래 메서드에서 확인해주세요.

            input_ohe = to_categorical(feat, num_classes=args.num_items)
            input_ohe = np.expand_dims(input_ohe, axis=1)
            target_ohe = to_categorical(target, num_classes=args.num_items)

            result = model.train_on_batch(input_ohe, target_ohe)
            tr_loader.set_postfix(train_loss=result[0], accuracy = result[1])

        val_recall, val_mrr = get_metrics(args.val, model, args, args.k)  # valid set에 대해 검증합니다.

        print(f"\t - Recall@{args.k} epoch {epoch}: {val_recall:3f}")
        print(f"\t - MRR@{args.k}    epoch {epoch}: {val_mrr:3f}\n")


def reset_hidden_states(model, mask):
    gru_layer = model.get_layer(name='GRU')  # model에서 gru layer를 가져옵니다.
    hidden_states = gru_layer.states[0].numpy()  # gru_layer의 parameter를 가져옵니다.
    for elt in mask:  # mask된 인덱스 즉, 종료된 세션의 인덱스를 돌면서
        hidden_states[elt, :] = 0  # parameter를 초기화 합니다.
    gru_layer.reset_states(states=hidden_states)


def get_metrics(data, model, args, k: int):  # valid셋과 test셋을 평가하는 코드입니다. 
                                             # train과 거의 같지만 mrr, recall을 구하는 라인이 있습니다.
    dataset = SessionDataset(data)
    loader = SessionDataLoader(dataset, batch_size=args.batch_size)
    recall_list, mrr_list = [], []

    total_step = len(data) - data['SessionId'].nunique()
    for inputs, label, mask in tqdm(loader, total=total_step // args.batch_size, desc='Evaluation', mininterval=1):
        reset_hidden_states(model, mask)
        input_ohe = to_categorical(inputs, num_classes=args.num_items)
        input_ohe = np.expand_dims(input_ohe, axis=1)

        pred = model.predict(input_ohe, batch_size=args.batch_size)
        pred_arg = tf.argsort(pred, direction='DESCENDING')  # softmax 값이 큰 순서대로 sorting 합니다.

        length = len(inputs)
        recall_list.extend([recall_k(pred_arg[i], label[i], k) for i in range(length)])
        mrr_list.extend([mrr_k(pred_arg[i], label[i], k) for i in range(length)])

    recall, mrr = np.mean(recall_list), np.mean(mrr_list)
    return recall, mrr

성능 평가는 recall과 mrr을 이용한다.  
recall은 우리가 많이 봤던 것이고 mrr은 무엇인가?!!  
[MRR (mean reciprocal rank)](https://ang-love-chang.tistory.com/22)


In [77]:
# 학습 시간이 다소 오래 소요됩니다. (예상시간 1시간)
train_model(model, args)

Train:   0%|          | 0/75 [00:00<?, ?it/s, accuracy=0, train_loss=5.01]     

	 - Recall@20 epoch 1: 0.161932
	 - MRR@20    epoch 1: 0.032135



Train:   0%|          | 0/75 [00:00<?, ?it/s, accuracy=0.0312, train_loss=5]   

	 - Recall@20 epoch 2: 0.170455
	 - MRR@20    epoch 2: 0.040201



Train:   0%|          | 0/75 [00:00<?, ?it/s, accuracy=0, train_loss=4.96]     

...

	 - Recall@20 epoch 28: 0.335227
	 - MRR@20    epoch 28: 0.113343



Train:   0%|          | 0/75 [00:00<?, ?it/s, accuracy=0.312, train_loss=3.04]

	 - Recall@20 epoch 29: 0.346591
	 - MRR@20    epoch 29: 0.114317



Evaluation:  61%|██████    | 11/18 [00:02<00:01,  5.18it/s]

	 - Recall@20 epoch 30: 0.349432
	 - MRR@20    epoch 30: 0.117801



## 6. Test

In [78]:

def test_model(model, args, test):
    test_recall, test_mrr = get_metrics(test, model, args, 20)
    print(f"\t - Recall@{args.k}: {test_recall:3f}")
    print(f"\t - MRR@{args.k}: {test_mrr:3f}\n")

test_model(model, args, test)

Evaluation:  11%|█         | 1/9 [00:00<00:01,  4.78it/s]

	 - Recall@20: 0.281250
	 - MRR@20: 0.093961



## try 2 : 삼년치 데이터 다쓰기

In [81]:
data_path = Path(os.getenv('HOME')+'/aiffel/yoochoose/data/') 
train_path = data_path / 'ratings.dat'

data = load_data(train_path, None)
data.sort_values(['UserId', 'Time'], inplace=True)  # data를 id와 시간 순서로 정렬해줍니다.
data['Time'] = data['Time'].map(lambda x: datetime.fromtimestamp(x))
data['delta'] = (data['Time'] - data['Time'].shift()).fillna(pd.Timedelta(seconds=0))
data['SessionId'] = data.apply(get_session_id, axis=1)
data = cleanse_recursive(data, shortest=2, logest=700, least_click=20) 
tr, test = split_by_date(data, n_days=180)   # 1년치가아니라 3년치 이기 때문에 바꿔 줘야 합니다.
tr, val = split_by_date(tr, n_days=180)
stats_info(tr, 'train')
stats_info(val, 'valid')
stats_info(test, 'test')

* train Set Stats Info
	 Events: 938973
	 Sessions: 16608
	 Items: 2992
	 First Time : 2000-04-25 23:05:32
	 Last Time : 2002-03-04 19:17:26

* valid Set Stats Info
	 Events: 11211
	 Sessions: 919
	 Items: 2450
	 First Time : 2002-03-04 20:36:36
	 Last Time : 2002-08-31 20:38:06

* test Set Stats Info
	 Events: 8022
	 Sessions: 612
	 Items: 2161
	 First Time : 2002-09-01 19:41:39
	 Last Time : 2003-02-28 17:49:50



In [85]:
# train set에 없는 아이템이 val, test기간에 생길 수 있으므로 train data를 기준으로 인덱싱합니다.
id2idx = {item_id : index for index, item_id in enumerate(tr['ItemId'].unique())}

tr = indexing(tr, id2idx)
val = indexing(val, id2idx)
test = indexing(test, id2idx)

save_path = data_path / 'processed'
save_path.mkdir(parents=True, exist_ok=True)

tr.to_pickle(save_path / 'train.pkl')
val.to_pickle(save_path / 'valid.pkl')
test.to_pickle(save_path / 'test.pkl')

tr_dataset = SessionDataset(tr)

tr_data_loader = SessionDataLoader(tr_dataset, batch_size=4)
iter_ex = iter(tr_data_loader)
inputs, labels, mask =  next(iter_ex)
print(f'Model Input Item Idx are : {inputs}')
print(f'Label Item Idx are : {"":5} {labels}')
print(f'Previous Masked Input Idx are {mask}')

Model Input Item Idx are : [ 0 40 53 65]
Label Item Idx are :       [ 1 41 54 62]
Previous Masked Input Idx are []


In [92]:
args = Args(tr, val, test, batch_size=512, hsz=50, drop_rate=0.1, lr=0.001, epochs=20, k=20)

In [93]:
model = create_model(args)

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(512, 1, 2992)]          0         
_________________________________________________________________
GRU (GRU)                    [(512, 50), (512, 50)]    456600    
_________________________________________________________________
dropout_6 (Dropout)          (512, 50)                 0         
_________________________________________________________________
dense_6 (Dense)              (512, 2992)               152592    
Total params: 609,192
Trainable params: 609,192
Non-trainable params: 0
_________________________________________________________________


In [94]:
train_model(model, args)

Train:   0%|          | 0/1801 [00:00<?, ?it/s, accuracy=0.00391, train_loss=7.28]

	 - Recall@20 epoch 1: 0.042969
	 - MRR@20    epoch 1: 0.013389



Train:   0%|          | 0/1801 [00:00<?, ?it/s, accuracy=0.0176, train_loss=6.91]

	 - Recall@20 epoch 2: 0.077637
	 - MRR@20    epoch 2: 0.021397



Train:   0%|          | 0/1801 [00:00<?, ?it/s, accuracy=0.0234, train_loss=6.75]

...

	 - Recall@20 epoch 18: 0.103027
	 - MRR@20    epoch 18: 0.033905



Train:   0%|          | 0/1801 [00:00<?, ?it/s, accuracy=0.041, train_loss=6.4]  

	 - Recall@20 epoch 19: 0.103027
	 - MRR@20    epoch 19: 0.034181



Evaluation:  20%|██        | 4/20 [00:11<00:44,  2.80s/it]

	 - Recall@20 epoch 20: 0.104980
	 - MRR@20    epoch 20: 0.034370



In [95]:
test_model(model, args, test)

Evaluation:   7%|▋         | 1/14 [00:02<00:35,  2.73s/it]

	 - Recall@20: 0.099609
	 - MRR@20: 0.035424



## try 3 :  2년치 데이터

In [96]:
data_path = Path(os.getenv('HOME')+'/aiffel/yoochoose/data/') 
train_path = data_path / 'ratings.dat'

data = load_data(train_path, None)
data.sort_values(['UserId', 'Time'], inplace=True)  # data를 id와 시간 순서로 정렬해줍니다.
data['Time'] = data['Time'].map(lambda x: datetime.fromtimestamp(x))
data['delta'] = (data['Time'] - data['Time'].shift()).fillna(pd.Timedelta(seconds=0))
data['SessionId'] = data.apply(get_session_id, axis=1)
month_ago = latest - dt.timedelta(days=730)     # 최종 날짜로부터 730일 이전 날짜를 구한다.  
data = data[data['Time'] > month_ago]   # 방금 구한 날짜 이후의 데이터만 모은다. 
data = cleanse_recursive(data, shortest=2, logest=700, least_click=20) 
tr, test = split_by_date(data, n_days=60)   # 1년치가아니라 3년치 이기 때문에 바꿔 줘야 합니다.
tr, val = split_by_date(tr, n_days=60)
stats_info(tr, 'train')
stats_info(val, 'valid')
stats_info(test, 'test')

* train Set Stats Info
	 Events: 43989
	 Sessions: 3470
	 Items: 1123
	 First Time : 2001-02-28 19:06:53
	 Last Time : 2002-10-31 16:33:08

* valid Set Stats Info
	 Events: 2225
	 Sessions: 173
	 Items: 911
	 First Time : 2002-11-01 02:56:19
	 Last Time : 2002-12-30 16:48:39

* test Set Stats Info
	 Events: 2363
	 Sessions: 191
	 Items: 934
	 First Time : 2002-12-30 18:21:02
	 Last Time : 2003-02-28 17:49:08



In [97]:
# train set에 없는 아이템이 val, test기간에 생길 수 있으므로 train data를 기준으로 인덱싱합니다.
id2idx = {item_id : index for index, item_id in enumerate(tr['ItemId'].unique())}

tr = indexing(tr, id2idx)
val = indexing(val, id2idx)
test = indexing(test, id2idx)

save_path = data_path / 'processed'
save_path.mkdir(parents=True, exist_ok=True)

tr.to_pickle(save_path / 'train.pkl')
val.to_pickle(save_path / 'valid.pkl')
test.to_pickle(save_path / 'test.pkl')

tr_dataset = SessionDataset(tr)

tr_data_loader = SessionDataLoader(tr_dataset, batch_size=4)
iter_ex = iter(tr_data_loader)
inputs, labels, mask =  next(iter_ex)
print(f'Model Input Item Idx are : {inputs}')
print(f'Label Item Idx are : {"":5} {labels}')
print(f'Previous Masked Input Idx are {mask}')

Model Input Item Idx are : [ 0  2 12 21]
Label Item Idx are :       [ 1  3 13 22]
Previous Masked Input Idx are []


In [104]:
args = Args(tr, val, test, batch_size=128, hsz=50, drop_rate=0.1, lr=0.001, epochs=20, k=20)

In [105]:
model = create_model(args)

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(128, 1, 1123)]          0         
_________________________________________________________________
GRU (GRU)                    [(128, 50), (128, 50)]    176250    
_________________________________________________________________
dropout_9 (Dropout)          (128, 50)                 0         
_________________________________________________________________
dense_9 (Dense)              (128, 1123)               57273     
Total params: 233,523
Trainable params: 233,523
Non-trainable params: 0
_________________________________________________________________


In [106]:
train_model(model, args)

Train:   0%|          | 0/316 [00:00<?, ?it/s, accuracy=0, train_loss=6.86]      

	 - Recall@20 epoch 1: 0.054688
	 - MRR@20    epoch 1: 0.008696



Train:   0%|          | 0/316 [00:00<?, ?it/s, accuracy=0.00781, train_loss=6.79]

	 - Recall@20 epoch 2: 0.058594
	 - MRR@20    epoch 2: 0.010301



Train:   0%|          | 0/316 [00:00<?, ?it/s, accuracy=0.00781, train_loss=6.56]

...

	 - Recall@20 epoch 18: 0.128906
	 - MRR@20    epoch 18: 0.047217



Train:   0%|          | 0/316 [00:00<?, ?it/s, accuracy=0.0938, train_loss=5.02]

	 - Recall@20 epoch 19: 0.128906
	 - MRR@20    epoch 19: 0.046937



Evaluation:  12%|█▎        | 2/16 [00:01<00:10,  1.39it/s]

	 - Recall@20 epoch 20: 0.125000
	 - MRR@20    epoch 20: 0.047875



In [107]:
test_model(model, args, test)

Evaluation:  12%|█▎        | 2/16 [00:01<00:10,  1.38it/s]

	 - Recall@20: 0.078125
	 - MRR@20: 0.036244



---
# 회고

- session을 직접 정의하는 부분이 재미가 있었다. session은 클라이언트 별로 서버에 저장되는 정보 인데 보톤 30분정도 유지 된다고 한다. 처음에는 그냥 같은 시간대만 모아가지고 했으나 뭔가 1초 단위 였기 때문에 성능이 매우 안좋았다. 그래서 시간을 위의 데이터와 시간 차를 구해서 30분 이상이면 새로운 세션아이디를 주는 것으로 처리하였다.

- 데이터를 1년치, 2년치, 3년치로 나눠서 한번 테스트를 해봤다   
1년치 : recall - 0.281250    MRR - 0.093961  
2년치 : recall - 0.099609    MRR - 0.035424   
3년치 : recall - 0.078125    MRR - 0.036244

- 가장 좋은 성능을 보인 것은 오히려 1년치 데이터 였다. 추천 시스템은 최근 데이터가 중요하다고 한다. 근데 영화는 뭔가 옛날 영화도 많이 찾아 본다고 생각해서 3년치로 데이터가 많으면 많을수록 좋을 거라고 생각했으나 1년치가 가장 좋은 성능을 보였다.

- least_click을 조금 올리는 것이 성능을 올린다고 한다!! ( 형준님 진아님 감사합니다)

- apply 함수를 쓰는거에 처음엔 미숙해서 for문으로 하려다가 너무 오래 걸려서 공부를해서 사용했다,,, 아직 완벽하게 알지는 못하지만 그래도 다음번에도 또 잘 사용할 수 있을 것 같다